In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Load the Key token
_ = load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

In [9]:
import sqlite3
import hashlib
import pandas as pd

conn = sqlite3.connect('database/employee_data.db')
cursor = conn.cursor()

email="xperry@gmail.com"
password="password123"

# Hash the input password
password_hash = hashlib.sha256(password.encode()).hexdigest()

# Query to find the employee with matching email and password hash
cursor.execute(
    "SELECT * FROM employees WHERE email=? AND password_hash=?",
    (email, password_hash),
)
employee = cursor.fetchone()

query = f"SELECT * FROM employees WHERE email = '{email}' AND password_hash = '{password_hash}'"

df = pd.read_sql(query, conn)

conn.close()

if employee:
    print(f"Authentication successful for: {employee[1]} {employee[2]}")
    employee_data = df.to_dict('records')

Authentication successful for: James Johnson


In [10]:
employee_data

[{'employee_id': 5,
  'first_name': 'James',
  'last_name': 'Johnson',
  'email': 'xperry@gmail.com',
  'phone_number': '620-638-0771x882',
  'department': 'IT',
  'position': 'Coordinator',
  'hire_date': '16/12/2021',
  'base_salary': 55650.27,
  'bonus': 9234.58,
  'currency': 'USD',
  'annual_leave_balance': 9,
  'sick_leave_balance': 4,
  'performance_rating': 2.61,
  'review_period': 'Q1 2024',
  'last_review_date': '26/05/2024',
  'password_hash': 'ef92b778bafe771e89245b89ecbc08a44a4e166c06659911881f383d4473e94f',
  'last_login': '26/08/2024 17:55'}]

In [12]:
user_query="How many leaves do i have?"

prompt = f"""
    Employee Information: {employee_data}
    
    Question: {user_query}
    """

system_message = """
You are an assistant helping non-technical employees with their questions using the information provided.

You will receive the data about the employee in a list of dictionary
the variables of the each dictionary are:
    employee_id: Unique identifier for each employee
    first_name: Employee's first name
    last_name: Employee's last name
    email: Employee's email (used for authentication)
    phone_number: Employee's contact number
    department: Employee's department
    position: Job title or position
    hire_date: Date when the employee was hired
    base_salary: Employee's base salary
    bonus: Most recent bonus amount
    currency: Currency of the salary
    annual_leave_balance: Number of annual leave days remaining
    sick_leave_balance: Number of sick leave days remaining
    performance_rating: Latest performance rating (e.g., 4.5)
    review_period: Period of the last review (e.g., Q1, 2024)
    last_review_date: Date of the last performance review
    password_hash: Hashed password for secure authentication
    last_login: Date and time of the last login
"""

In [13]:
# Send the prompt to the LLM model
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": system_message,
        },
        {"role": "user", "content": prompt},
    ],
)

In [15]:
# Return the LLM's response
response.choices[0].message.content

'You have a total of 9 annual leave days remaining and 4 sick leave days remaining.'

In [16]:
import time

In [18]:
all_masseges = list()
all_prompts = list()
model_responses = list()

# Define system promt
all_masseges.append({"role": "system", "content": system_message})

while True:
    
    user_question = input('You: ')
    
    
    if user_question.lower() in ['quit', 'exit', 'ex', 'escape', 'bye']:
      print('Thank you for using our services')
      time.sleep(2)
      break
    
    elif user_question == '':
      continue
    
    else:
      
      user_promt = f"""
      Employee Information: {employee_data}
      
      Question: {user_question}
      """
      
      # append user promt
      all_prompts.append(user_promt)
      
      # append in the messages list of dicts
      all_masseges.append({"role": "user", "content": user_promt})
      
      # model
      each_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=all_masseges
        )
      
      each_response = each_response.choices[0].message.content
      print(f'ChatBot: {each_response}')
      
      # append response
      model_responses.append(each_response)
      
      # append AI response
      all_masseges.append({"role": "assistant", "content": each_response})

ChatBot: You have 4 sick leave days remaining.
ChatBot: Could you please clarify what you mean by "the total"? Are you referring to your total annual leave, sick leave, salary, or something else?
ChatBot: If you are asking for the total of your annual leave and sick leave balances, you have a total of 13 leave days (9 annual leave days + 4 sick leave days). 

If you meant something else, please let me know!
ChatBot: Your base salary is $55,650.27 USD.
ChatBot: Yes, you have a bonus of $9,234.58 USD.
ChatBot: Yes, you have a bonus of $9,234.58 USD.
Thank you for using our services
